In [ ]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
from time import time

"""
PointEnv from rllab
The goal is to control an agent and get it to the target located at (0,0).
At each timestep the agent gets its current location (x,y) as observation, 
takes an action (dx,dy), and is transitioned to (x+dx, y+dy).
"""

class PointEnv():
    def reset(self):
        self._state = np.random.uniform(-1, 1, size=(2,))
        state = np.copy(self._state)
        return state

    def step(self, action):
        action = np.clip(action, -1, 1)
        self._state = self._state + 0.1 * action
        x, y = self._state
        reward = -(x ** 2 + y ** 2) ** 0.5 - 0.02 * np.sum(action ** 2)
        done = abs(x) < 0.01 and abs(y) < 0.01
        next_state = np.copy(self._state)
        return next_state, reward, done

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import numpy as np

# make these smaller to increase the resolution
dx, dy = 0.05, 0.05

# generate 2 2d grids for the x & y bounds
y, x = np.mgrid[slice(-5, 5 + dy, dy),
                slice(-5, 5 + dx, dx)]

z = -np.sqrt(x**2 + y**2)

# x and y are bounds, so z should be the value *inside* those bounds.
# Therefore, remove the last value from the z array.
z = z[:-1, :-1]
levels = MaxNLocator(nbins=15).tick_values(z.min(), z.max())


# pick the desired colormap, sensible levels, and define a normalization
# instance which takes data values and translates those into levels.
cmap = plt.get_cmap('spring')
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

fig, ax0 = plt.subplots(nrows=1)

im = ax0.pcolormesh(x, y, z, cmap=cmap, norm=norm)
fig.colorbar(im, ax=ax0)
ax0.set_title('Reward')

plt.show()

### Continuous Policy Gradient Theorem

Let's examine the policy gradient theorem for continuous domains. Recall the reinforcement learning objective, which we write here in full:
\begin{align}
J(\theta)=\int p_0(s)\int \pi_\theta(a\vert s) Q(a,s) da ds. \label{eq:rl_objective}
\end{align}
Just like in the discrete case (https://papers.nips.cc/paper/1713-policy-gradient-methods-for-reinforcement-learning-with-function-approximation.pdf , (Sutton et al. 2000)), we can take derivatives of objective with respect to the policy parameters $\theta$, yielding:
\begin{align}
\nabla_\theta J(\theta)=\int\pi_\theta(a_t\vert s_t)\int \nabla_\theta\pi_\theta(a\vert s) Q(a,s) da ds ,\label{eq:policy_gradient_theorem}
\end{align} 
where $\pi_\theta(a_t\vert s_t)$ is the discounted ergodic occupancy measure. Using the log-derivative trick, we write \cref{eq:policy_gradient_theorem} as an expectation over actions:
\begin{align}
\nabla_\theta J(\theta)=&\int\pi_\theta(a_t\vert s_t)\int\pi_\theta(a\vert s) \nabla_\theta\log\pi_\theta(a\vert s) Q(a,s) da ds,\\ =&\mathbb{E}_{s\sim\pi_\theta(a_t\vert s_t),a\sim\pi_\theta(a\vert s)}\Big[\nabla_\theta\log\pi_\theta(a\vert s) Q(a,s)\Big].
\end{align}
We then approximate the expectation by using $N$ samples from the environment under our policy $\pi_\theta(a\vert s)$, giving:
\begin{align}
\nabla_\theta J(\theta)\approx&\frac{1}{N}\sum_{n=0}^{N-1}\sum_{t=0}^{T_N}\nabla_\theta\log\pi_\theta(a_t\vert s_t) Q(a_t,s_t).
\end{align}


### REINFORCE and Baselines

Using a Monte-Carlo approximator for the action-value function, $Q(a_t,s_t)\approx R_t\triangleq\sum_{i=t}^T r(a_i,s_i)$, we recover the REINFORCE (http://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf ,(Williams 1992)) algorithm:
\begin{align}
\nabla_\theta J(\theta)\approx&\frac{1}{N}\sum_{n=0}^{N-1}\sum_{t=0}^{T_N}\nabla_\theta\log\pi_\theta(a_t\vert s_t) R_t.
\end{align}

### Exercise I.1: Implementing REINFORCE

We choose our policy to be a linear Gaussian policy with parameters $\theta$. Given state $s$, we can define some features $\phi(s)$ and sample an action $a \sim N(\phi(s)^T \theta, \sigma^2)$. Our PointEnv environment is simple enough that we can use $\phi(s) = s$. Note that $\sigma^2$ can also depend on $s$, but we have kept it constant in Part I for simplicity. Policy gradient algorithms use the update rule $\theta' = \theta + \alpha \nabla_\theta J(\pi)$, where $\alpha$ is the learning rate and $J(\pi)$ is the expected return of the policy.

PROBLEM I.1) In the function "get\_action\_and\_grad()", sample an action from $\pi_\theta(a\vert s)$ and compute the corresponding value of $\nabla_\theta\log\pi_\theta(a\vert s)$.

In [ ]:
class Gauss_Policy():
    def __init__(self):
        self.action_dim = 2
        self.theta = 0.5 * np.ones(4)
        # theta here is a length 4 array instead of a matrix for ease of processing
        # Think of treating theta as a 2x2 matrix and then flatenning it, which gives us:
        # action[0] = state[0]*[theta[0], theta[1]]
        # action[1] = state[1]*[theta[2], theta[3]]

    def get_action_and_grad(self, state):
        # Exercise I.1:
        mean_act = ...
        sampled_act = ...
        grad_log_pi = ...
        return sampled_act, grad_log_pi

In [ ]:
# This function collects some trajectories, given a policy
def gather_paths(env, policy, num_paths, max_ts=500):
    paths = []
    for i in range(num_paths):
        ts = 0
        states = []
        act = []
        grads = []
        rwd = []
        done = False
        s = env.reset()
        while not done and ts < max_ts:
            a, grad_a = policy.get_action_and_grad(s)
            next_s, r, done = env.step(a)
            states += [s]
            act += [a]
            rwd += [r]
            grads += [grad_a]
            s = next_s
            ts += 1
        path = {'states': np.array(states),
                'actions': np.array(act),
                'grad_log_pi': np.array(grads),
                'rwd': np.array(rwd)}
        paths += [path]
    return paths

### Exercise I.2: Using Baselines to Reduce Variance

We can subtract a baseline $V(s)$ from the action-value function, which does not affect the overall gradient, but will reduce variance in the gradient update. To see why, as $V_\omega(s)\triangleq\mathbb{E}_{a\sim\pi_\theta(a\vert s)}[Q_\omega(a,s)]$, the quantity $Q_\omega(a,s)-V\omega(s)$ will only be positive for $Q_\omega(a,s)>V_\omega(s)$, hence the gradient updates will therefore only be scaled by a positive value when an action has greater reward than the average return. For these reasons, we call $A_\omega(a,s)=Q_\omega(a,s)-V_\omega(s)$ the advantage. For REINFORCE, $A_\omega(a_t,s_t)=R_t-V_\omega(s_t)$. Just like we define features $\phi(s)$ for the policy, we can define some features $\psi(s,t)$ for the value function. Our features are going to be $\phi(s,t)=[s, s^2, t, t^2, t^3, 1]$ and we will approximate the value function as a linear function of these features, $V_\omega (s_t)=\omega^T\phi(s,t)$

PROBLEM I.2a) Given some sampled trajectories, fitting the value function is a linear regression problem.  The targets for our linear regression problem will therefore be the returns. The features have been implemented in the function "baselines()" you are required to compute the regression coefficients. Hint: The function "np.linalg.lstsq" may be of use.

PROBLEM I.2b) Now calculate the value for each state in \code{path} using the newly learnt coefficients and save it to "path[`value']"

In [ ]:
def baseline(paths):
    path_features = []
    for path in paths:
        s = path["states"]
        l = len(path["rwd"])
        al = np.arange(l).reshape(-1, 1) / 100.0
        path_features += [np.concatenate([s, s ** 2, al, al ** 2, al ** 3, np.ones((l, 1))], axis=1)]
    ft = np.concatenate([el for el in path_features])
    targets = np.concatenate([el['returns'] for el in paths])

    # Exercise I.2(a): Compute the regression coefficents
    coeffs = ...
    # Exercise I.2(b): Calculate the values for each state
    for i, path in enumerate(paths):
        path['value'] = ...

In [ ]:
def process_paths(paths, discount_rate=1):
    grads = []
    for path in paths:
        # Exercise I.3a: Implement the discounted return
        # Hint: This can be done in one line using lfilter from scipy.signal,
        # but it might be much easier to write a separate function for this
        path['returns'] = scipy.signal.lfilter([1], [1, float(-discount_rate)], path['rwd'][::-1], axis=0)[::-1]
    baseline(paths)
    for path in paths:
        path['adv'] = path['returns'] - path['value']
        rets_for_grads = np.atleast_2d(path['adv']).T
        rets_for_grads = np.repeat(rets_for_grads, path['grad_log_pi'].shape[1], axis=1)
        path['grads'] = path['grad_log_pi'] * rets_for_grads
        grads += [np.sum(path['grads'], axis=0)]
    grads = np.sum(grads, axis=0) / len(paths)
    return grads


In [ ]:
# Run algo
env = PointEnv()
alpha = 0.01
traj_len = 50
perf_stats = []

def run_algo(env, alpha, gamma, traj_len, num_itr=200, runs=10):
    rwd = np.zeros((num_itr, runs))
    for st in range(runs):
        policy = Gauss_Policy()
        for i in range(num_itr):
            paths = gather_paths(env, policy, max_ts=traj_len, num_paths=5)
            rwd[i, st] = np.mean([np.sum(path['rwd']) for path in paths])
            grads = process_paths(paths, discount_rate=gamma)
            policy.theta += alpha * grads
    perf_stats = {'gamma': gamma,
                  'mean_rwd': np.mean(rwd, axis=1),
                  'std_err': np.std(rwd, axis=1) / np.sqrt(runs)}
    return perf_stats

gamma = [0.99, 0.995, 1.0]
for g in gamma:
    print("Starting algorithm with gamma:", g)
    perf_stats += [run_algo(env, alpha, gamma=g, traj_len=traj_len)]

# And plot the results
for el in perf_stats:
    plt.plot(el['mean_rwd'], label='discount factor = ' + str(el['gamma']))
    plt.fill_between(np.arange(len(el['mean_rwd'])), el['mean_rwd'] + el['std_err'], el['mean_rwd'] - el['std_err'],
                     alpha=0.3)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Returns')
plt.xlim([0, 200])
plt.show()

# Exercise I.3(b): Run the algo again, but with traj_len=500.
# Does the relative performance of learning using discount factors change?